In [85]:
import pandas as pd
import pygwalker as pyg
from matplotlib import pyplot as plt

In [86]:
#read the data and show the original doctor data
doctor_data = pd.read_csv('output.csv',encoding='utf-8')
elder_data = pd.read_csv('regional_elder.csv',encoding='utf-8')
pyg.walk(doctor_data)

Box(children=(HTML(value='\n<div id="ifr-pyg-00061e5d48674266DxtsNMjfq4QenV9J" style="height: auto">\n    <hea…

In [87]:
#doctor data preprocessing, delete 2024 and synchronize the column index with the elder data
doctor_data = doctor_data.drop(columns=['2024.1/4'])


In [88]:
#original elder data
pyg.walk(elder_data)

Box(children=(HTML(value='\n<div id="ifr-pyg-00061e5d487fb6e3Ur8imKDGxqfd0pzO" style="height: auto">\n    <hea…

In [89]:
#only save the regional info, and only save the data collected elders who older than 60 ages
column_to_save = ['행정구역별(시군구)','2020','2021','2022','2023']
elder_data = elder_data[column_to_save]

#delete the 행정구역별(시군구),읍부,면부,동부
rows_to_drop = [0,2,3,4] 
elder_data = elder_data.drop(index=rows_to_drop)

# 将字符串转换为整数
for col in elder_data.columns[1:]:  # 忽略第一列 '행정구역별(시군구)'
    elder_data[col] = elder_data[col].astype(int)

pyg.walk(elder_data)

Box(children=(HTML(value='\n<div id="ifr-pyg-00061e5d489d8a6doYcFyA4l8DwrOS1f" style="height: auto">\n    <hea…

In [90]:
#confirm the index of 2 different dataset, and confirm the shape of the data
elder_data.shape
print(elder_data.index)
print(doctor_data.index)

Index([1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], dtype='int64')
RangeIndex(start=0, stop=18, step=1)


In [91]:
#mapping dictionary
number_to_location = {
    0: '전국',
    1: '서울특별시',
    2: '부산광역시',
    3: '대구광역시',
    4: '인천광역시',
    5: '광주광역시',
    6: '대전광역시',
    7: '울산광역시',
    8: '세종특별자치시',
    9: '경기도',
    10: '강원특별자치도',
    11: '충청북도',
    12: '충청남도',
    13: '전라북도',
    14: '전라남도',
    15: '경상북도',
    16: '경상남도',
    17: '제주특별자치도'
}


In [92]:
#now set the 1st column of elder dataset as the index, and synthesize the doctor dataset same as the elder dataset
#change column index of doctor df
doctor_data.columns = ['2020', '2021','2022','2023']
#change doctor data index from digits to the location
doctor_data.index = doctor_data.index.map(number_to_location)

#change the elder data 1st column into index
elder_data.set_index('행정구역별(시군구)', inplace=True)
# 打印转换后的 DataFrame
print("\ndoctor data with location names as index:")
print(doctor_data.index)
print("\nelder data with location names as index:")
print(elder_data.index)
print(doctor_data.index==elder_data.index)


doctor data with location names as index:
Index(['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'],
      dtype='object')

elder data with location names as index:
Index(['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'],
      dtype='object', name='행정구역별(시군구)')
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [98]:
print(f'{doctor_data=}\n{elder_data=}')

doctor_data=           2020    2021    2022    2023
전국       248635  255715  261238  267572
서울특별시     62934   64309   65523   67079
부산광역시     18745   19107   19473   20030
대구광역시     13606   14060   14425   14742
인천광역시     12516   13066   13311   13984
광주광역시      8944    9064    9145    9348
대전광역시      8624    8768    8856    8927
울산광역시      4340    4436    4536    4638
세종특별자치시     910    1149    1228    1287
경기도       53292   55550   57760   60093
강원특별자치도    6122    6366    6525    6450
충청북도       6543    6622    6769    6798
충청남도       8251    8370    8420    8511
전라북도       9186    9383    9445    9560
전라남도       8261    8431    8549    8504
경상북도       9536    9645    9721    9797
경상남도      13949   14427   14563   14758
제주특별자치도    2876    2962    2989    3066
elder_data=                2020      2021      2022      2023
행정구역별(시군구)                                        
전국          11623264  12265601  12809657  13319909
서울특별시        2124510   2206571   2279066   2349553
부산광역시        

In [103]:
assert doctor_data.shape == elder_data.shape, "DataFrames must have the same shape"

# 清洗和计算数据
def clean_and_divide(df1, df2):
    # 创建结果 DataFrame 的副本
    result = df1.copy()
    
    # 处理每一列
    for column in df1.columns:
        result[column] = df1[column] / df2[column].replace(0, pd.NA)
    
    return result

# 使用函数计算
result = clean_and_divide(elder_data, doctor_data)

# 将索引转移到 DataFrame 的第一列
result = result.reset_index()
result = result.rename(columns={'index': 'First_Column'})

pyg.walk(result)

Box(children=(HTML(value='\n<div id="ifr-pyg-00061e5d5b913b97OFDQUqzlw49dJLEe" style="height: auto">\n    <hea…